# Modelling Selected Parameters at 0:00 + Day Phase

## Model Selection

In [1]:
import os
import pandas as pd
import numpy as np

data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
df = pd.read_csv(data_file, index_col=0)
df.head()

/var/folders/g7/z7m625rj50d5ptztzplh4wgc0000gn/T/ipykernel_5112/33917844.py:6: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file, index_col=0)


,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
id,,,,,,,,,,,,,,,,,,,,,
p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


# Data Preprocessing

In [2]:
from pipelines import preprocessing_pipeline

df = preprocessing_pipeline.fit_transform(df)
df.head()

,day_phase,bg-0:00,insulin-0:00,hr-0:00,cals-0:00,bg+1:00
id,,,,,,
p01_0,morning,15.1,0.0417,99.6,4.8,13.4
p01_1,morning,14.4,0.0417,99.6,4.8,12.8
p01_2,morning,13.9,0.0417,99.6,4.8,15.5
p01_3,morning,13.8,0.0417,99.6,4.8,14.8
p01_4,morning,13.4,0.0417,99.6,4.8,12.7


# Model selection

## 1. Split the data into train and test sets

In [3]:
from sklearn.model_selection import train_test_split
from pipelines import standardization_pipeline

X = df.drop(columns=['bg+1:00'])
y = df['bg+1:00']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the data
X_train = standardization_pipeline.fit_transform(X_train)
X_test = standardization_pipeline.fit_transform(X_test)

X_train.head()

,bg-0:00,insulin-0:00,hr-0:00,cals-0:00,day_phase_evening,day_phase_morning,day_phase_night,day_phase_noon
id,,,,,,,,
p12_24040,-0.826509,-0.069842,-0.532922,-0.201556,False,False,False,True
p12_13267,-0.459549,-0.193942,-0.809441,-0.201556,False,True,False,False
p03_131,1.875652,-0.003623,-0.660940,-0.852188,False,False,False,False
p12_20861,-0.993309,-0.193942,1.489761,0.652399,False,False,False,False
p12_24627,-0.426189,-0.193942,0.870154,0.066043,False,False,False,False


## 2. Use LazyPredict

In [4]:
from notebooks.helpers.LazyPredict import get_lazy_regressor

reg = get_lazy_regressor()
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 38/39 [14:10<00:48, 48.38s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000635 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 975
[LightGBM] [Info] Number of data points in the train set: 141619, number of used features: 4
[LightGBM] [Info] Start training from score 8.276012


100%|██████████| 39/39 [14:10<00:00, 21.81s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
LGBMRegressor,0.52,0.52,2.08,0.55
HistGradientBoostingRegressor,0.52,0.52,2.08,0.83
XGBRegressor,0.51,0.51,2.11,0.35
GradientBoostingRegressor,0.51,0.51,2.12,7.54
MLPRegressor,0.50,0.50,2.12,21.36
SVR,0.49,0.49,2.15,471.97
PoissonRegressor,0.48,0.48,2.18,0.02
BayesianRidge,0.48,0.48,2.18,0.04
RidgeCV,0.48,0.48,2.18,0.03


The best models are:

* HistGradientBoostingRegressor
* LGBMRegressor
* GradientBoostingRegressor

For further work I choose:

* HistGradientBoostingRegressor
* XGBRegressor
* RidgeCV

## Save the train data for later

In [6]:
# standardize the whole data
df = standardization_pipeline.fit_transform(X=df)
df.to_csv('train_data.csv', index=False)